In [ ]:
class Convert2RGB(object):
  def __init__(self,num_channel):
    self.num_channel=num_channel
  
  def __call__(self,img):
    img_channel=img.shape[0]
    img=torch.cat([img]*(self.num_channel-img_channel+1),0)
    return img

In [ ]:
class Pad(object):
  def __init__(self,size,fill=0,padding_mode='constant'):
    self.size=size
    self.fill=fill
    self.padding_mode=padding_mode

  def __call__(self,img):
    img_size=img.shape[1]
    assert ((self.size-img_size)%2==0)
    padding=(self.size-img_size)//2
    padding=(padding,padding,padding,padding)
    return F.pad(img,padding,self.padding_mode,self.fill)

In [ ]:
def get_transform():
  transform=transforms.Compose([
      transforms.ToTensor(),
      Pad(32),
      Convert2RGB(3),
      transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])
  ])
  return transform

In [ ]:
def save_model(mode,optimizer,store_model_path):
  torch.save(model.state_dict(),f'{store_model_path}.ckpt')
  torch.save(optimizer.state_dict(),f'{store_model_path}.opt')
  return

In [ ]:
def load_model(model,optimizer,load_model_path):
  print(f'Load model from {load_model_path}')
  model.load_state_dict(torch.load(f'{load_model_path}.ckpt'))
  optimizer.load_state_dict(torch.load(f'{load_model_path}.opt'))

In [ ]:
def build_model(data_path,batch_size,learning_rate):
  model=Model().to(device)
  optimizer=optim.Adam(model.parameters(),lr=learning_rate)
  data=Data(data_path)
  datasets=data.get_datasets()
  tasks=[]
  for dataset in datasets:
    tasks.append(DataLoader(dataset,batch_size))
  return model,optimizer,tasks

In [ ]:
# 产生npoints个维度为ndim的单位球壳向量
def sample_spherical(npoints,ndim=3):
  vec=np.random.randn(ndim,npoints)  
  vec/=np.linalg.norm(vec,axis=0)  
  return vec